In [ ]:
import numpy as np
import pandas as pd
from keras.layers import *
from keras.models import *
from keras.preprocessing import image
from keras.metrics import accuracy, binary_crossentropy
from keras.optimizers import Adam
from keras.regularizers import l2
import os

In [ ]:
if (not os.path.exists("CovidDataset.zip")):
  # Download the dataset file from dropbox
  !wget https://www.dropbox.com/s/tlsdn617iymz3bf/CovidDataset.zip
if (not os.path.exists("Dataset/")):
  # Unzip the filed
  !unzip CovidDataset.zip

In [ ]:
TRAIN_PATH = "DataSet/Train"
VAL_PATH = "DataSet/Val"

In [ ]:
# Build CNN Based Model in Keras
model = Sequential()
model.add(Conv2D(32, kernel_size = (3,3), activation = 'relu', input_shape = (224, 224, 3)))
model.add(Conv2D(64, (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(64, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation = 'sigmoid'))

In [ ]:
# Compile the model
model.compile(loss = "binary_crossentropy", 
              optimizer = 'adam', 
              metrics = ['accuracy'])

In [ ]:
# Summary of the Model
model.summary()

In [ ]:
# Processing the images
train_datagen = image.ImageDataGenerator(
    rescale = 1/255.,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True
)

val_datagen = image.ImageDataGenerator(rescale = 1/255.)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    'Dataset/Train',
    target_size = (224, 224),
    batch_size = 32,
    class_mode = 'binary'
)

In [ ]:
val_generator = val_datagen.flow_from_directory(
    'Dataset/Val',
    target_size = (224, 224),
    batch_size = 32,
    class_mode = 'binary'
    
)

In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch = 8,
    epochs = 20,
    validation_data = val_generator,
    validation_steps = 2
)

In [ ]:
# Evaluate the model
model.evaluate(val_generator)

In [ ]:
train_acc = round(history.history['accuracy'][-1], 2) * 100
val_acc = round(history.history['val_accuracy'][-1], 2) * 100

In [ ]:
print(f"The Training accuracy is {train_acc * 100}%")
print(f"The validation accuracy is {val_acc * 100}%")

In [ ]:
model.save(f"cnn-cxr-acc-{val_acc}_bs-32_epochs-20.h5")
print(f"Model saved with {val_acc} % accuracy.")

In [ ]:
# Modify 'test1.jpg' and 'test2.jpg' to the images you want to predict on
from keras.models import load_model
from keras.preprocessing import image
import numpy as np

def classify_cxr(img_path):
  # dimensions of our images
  img_width, img_height = 224, 224

  # load the model we saved
  model = load_model('cnn-cxr-acc-98.44_bs-32_epochs-20.h5')
  model.compile(loss='binary_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])

  # predicting images
  img = image.load_img(img_path, target_size = (img_width, img_height))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  predict_class = model.predict(images)
  return predict_class[0][0]

img_base_path = "Dataset/Train/Normal/"
img_names = os.listdir(img_base_path)
img_path = img_base_path + img_names[10]

classified = classify_cxr(img_path)

if (int(classified) == 0):
  print(f"Reslut: Covid Chest XRay")
else:
  print(f"Reslut: Non-Covid Chest XRay")

In [ ]:
!pip freeze > requirements.txt